In [1]:
rows = []
routes = open("routes.txt", "r+")
for row in routes:
    row = row.rstrip()
    row = row.split()
    try:
        row = [int(r) for r in row]
    except:
        pass
    rows.append(row)
print(rows)

[[1, 2, 2], [1, 3, 6], [1, 4, 7], [1, 5, 2], [2, 4, 10], [2, 5, 2], [3, 5, 2], [4, 6, 3], [5, 6, 1]]


In [2]:
# Finding out the nodes from the list of nodes and links

import numpy as np
import itertools

array = np.array(rows)
all_nodes = array[:, :2]
nodes = sorted(set(itertools.chain(*all_nodes)))
try:
    nodes = [int(node) for node in nodes]
except:
    pass
nodes_count = len(nodes)
node_index = {node: idx for idx, node in enumerate(nodes)}
print(f"Nodes: {nodes}")
print(f"Nodes count: {len(nodes)}")
print(f"Node order: {node_index}")

Nodes: [1, 2, 3, 4, 5, 6]
Nodes count: 6
Node order: {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5}


In [3]:
#nodes_count = 6
all_routes = []
for i in nodes:
    routes = []
    for row in rows:
        if i == row[0]:     # checking if node matches with the first element in the list
            dnc = [row[1], row[1], row[2]]      # (destination node, next node, link cost)
            routes.append(dnc)
            
        elif i == row[1]:   # checking if node matches with the second element in the list
            dnc = [row[0], row[0], row[2]]      # (destination node, next node, link cost)
            routes.append(dnc)
            
    all_routes.append(routes)  
print(all_routes)

[[[2, 2, 2], [3, 3, 6], [4, 4, 7], [5, 5, 2]], [[1, 1, 2], [4, 4, 10], [5, 5, 2]], [[1, 1, 6], [5, 5, 2]], [[1, 1, 7], [2, 2, 10], [6, 6, 3]], [[1, 1, 2], [2, 2, 2], [3, 3, 2], [6, 6, 1]], [[4, 4, 3], [5, 5, 1]]]


In [4]:
# The list "all_routes" will have the routing table of nodes as given above in this cell
'''all_routes = [[[2, 2, 2],
                [3, 3, 6],
                [4, 4, 7],
                [5, 5, 2]],
                
               [[1, 1, 2],
                [4, 4, 10],
                [5, 5, 2]],
                
               [[1, 1, 6],
                [5, 5, 2]],
        
               [[1, 1, 7],
                [2, 2, 10],
                [6, 6, 3]],
            
               [[1, 1, 2],
                [2, 2, 2],
                [3, 3, 2],
                [6, 6, 1]],
                
               [[4, 4, 3],
                [5, 5, 1]]]'''

# Printing the initial routing tables of the nodes in a serial fashion (from 1 to 6 in this case)
for i in range(nodes_count):
    print(f"routing table of node {i+1} = {all_routes[i]}")

routing table of node 1 = [[2, 2, 2], [3, 3, 6], [4, 4, 7], [5, 5, 2]]
routing table of node 2 = [[1, 1, 2], [4, 4, 10], [5, 5, 2]]
routing table of node 3 = [[1, 1, 6], [5, 5, 2]]
routing table of node 4 = [[1, 1, 7], [2, 2, 10], [6, 6, 3]]
routing table of node 5 = [[1, 1, 2], [2, 2, 2], [3, 3, 2], [6, 6, 1]]
routing table of node 6 = [[4, 4, 3], [5, 5, 1]]


In [5]:
from copy import deepcopy
import time

connected_nodes = []
for i in range(nodes_count):
    table = all_routes[i]
    # append the connected nodes such as nodes 2, 3, 4 and 5 are connected to node 1 and so on 
    connected_nodes.append([row[0] for row in table])
print(connected_nodes)

[[2, 3, 4, 5], [1, 4, 5], [1, 5], [1, 2, 6], [1, 2, 3, 6], [4, 5]]


In [6]:
def update_routes(nodes, connected_nodes, all_routes):
    all_routes = deepcopy(all_routes)
    for i, node in enumerate(nodes):
        for n in connected_nodes[i]:
            reference_table = all_routes[node_index.get(n)]
            
            for row in reference_table:
                if row[0] == node:         # finding current node in reference table
                    base_dist = row[2] 
                    break
                    
            for row in reference_table:        
                if row[0] != node:         # finding distance to nodes other than the current node
                    temp_dist = base_dist + row[2]
                    
                    table_to_update = all_routes[i]
                    for k, table_row in enumerate(table_to_update):
                        # check if node in reference routing table of connected node is in the routing table of the current node
                        if row[0] == table_row[0]:
                            if temp_dist < table_row[2]:
                                table_row[2] = temp_dist
                                table_row[1] = n
                            break
                                
                        elif k == len(table_to_update) - 1:
                            table_to_update.append([row[0], n, temp_dist])
                            
        print(all_routes)
        print("\n")
        
    return all_routes

In [7]:
iteration = 1
start = time.time()

while True:
    print(f"Iteration {iteration}")
    print("-----------------------------------------------")
    prev_routes = deepcopy(all_routes)
    all_routes = update_routes(nodes, connected_nodes, all_routes)
    
    # Check if previous routing tables are equal to the updated routing tables
    if prev_routes == all_routes:
        print("Well, Yeah!")
        end = time.time()
        print("Total time =", end - start)
        break
        
    else:
        print("### Let's try this again! ###")
        iteration += 1

Iteration 1
-----------------------------------------------
[[[2, 2, 2], [3, 5, 4], [4, 4, 7], [5, 5, 2], [6, 5, 3]], [[1, 1, 2], [4, 4, 10], [5, 5, 2]], [[1, 1, 6], [5, 5, 2]], [[1, 1, 7], [2, 2, 10], [6, 6, 3]], [[1, 1, 2], [2, 2, 2], [3, 3, 2], [6, 6, 1]], [[4, 4, 3], [5, 5, 1]]]


[[[2, 2, 2], [3, 5, 4], [4, 4, 7], [5, 5, 2], [6, 5, 3]], [[1, 1, 2], [4, 1, 9], [5, 5, 2], [3, 5, 4], [6, 5, 3]], [[1, 1, 6], [5, 5, 2]], [[1, 1, 7], [2, 2, 10], [6, 6, 3]], [[1, 1, 2], [2, 2, 2], [3, 3, 2], [6, 6, 1]], [[4, 4, 3], [5, 5, 1]]]


[[[2, 2, 2], [3, 5, 4], [4, 4, 7], [5, 5, 2], [6, 5, 3]], [[1, 1, 2], [4, 1, 9], [5, 5, 2], [3, 5, 4], [6, 5, 3]], [[1, 5, 4], [5, 5, 2], [2, 5, 4], [4, 1, 11], [6, 5, 3]], [[1, 1, 7], [2, 2, 10], [6, 6, 3]], [[1, 1, 2], [2, 2, 2], [3, 3, 2], [6, 6, 1]], [[4, 4, 3], [5, 5, 1]]]


[[[2, 2, 2], [3, 5, 4], [4, 4, 7], [5, 5, 2], [6, 5, 3]], [[1, 1, 2], [4, 1, 9], [5, 5, 2], [3, 5, 4], [6, 5, 3]], [[1, 5, 4], [5, 5, 2], [2, 5, 4], [4, 1, 11], [6, 5, 3]], [[1, 1, 7], [